
# Value at Risk (VaR) - 历史模拟法演示

📌 本 notebook 用于演示如何使用 Python 进行 VaR（风险价值）计算，
以等权重的债券ETF组合为例，利用历史模拟法估算 1-day 95% 的 VaR。


In [ ]:

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 设置绘图风格
plt.style.use('seaborn-vintage')



## 1️⃣ 设置组合成分

- EMHY: iShares J.P. Morgan EM High Yield Bond ETF  
- HYEM: VanEck EM High Yield Bond ETF  
- EMB: iShares J.P. Morgan USD EM Bond ETF  
- PCY: Invesco EM Sovereign Debt ETF  
- EMLC: VanEck J.P. Morgan EM Local Currency Bond ETF


In [ ]:

tickers = ['EMHY', 'HYEM', 'EMB', 'PCY', 'EMLC']


## 2️⃣ 下载近两年的日收盘价

In [ ]:

raw = yf.download(tickers, start='2022-08-01', end='2025-07-01')
prices = raw['Close']
prices.columns.name = None  # 移除列名多层索引，使 DataFrame 更清晰
prices.head()


## 3️⃣ 检查缺失值并处理

In [ ]:

if prices.isna().sum().sum() > 0:
    print("⚠️ 存在缺失值，进行前向填充...")
    prices = prices.fillna(method='ffill')


## 4️⃣ 计算每日对数收益率

In [ ]:

returns = prices.pct_change().dropna()
returns.head()


## 5️⃣ 设置组合权重并计算组合收益

In [ ]:

weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
portfolio_ret = returns.dot(weights)
portfolio_ret.head()


## 6️⃣ 使用历史模拟法计算 VaR

In [ ]:

confidence = 0.95
var_95 = np.percentile(portfolio_ret, (1 - confidence) * 100)

portfolio_value = 10_000_000  # 示例：1000 万美元
dollar_var = portfolio_value * var_95

print(f"\n📉 1-day 95% VaR (历史模拟法): {var_95:.4%}")
print(f"💰 名义 VaR（对应 1000 万美元组合）: ${dollar_var:,.0f}\n")


## 7️⃣ 可视化：每日收益率分布与 VaR

In [ ]:

plt.figure(figsize=(10,6))
plt.hist(portfolio_ret, bins=60, color='steelblue', alpha=0.7)
plt.axvline(var_95, color='red', linestyle='--', label=f'VaR 95% = {var_95:.2%}')
plt.title('📊 Portfolio Daily Return Distribution & VaR', fontsize=14)
plt.xlabel('Daily Return')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
